In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch.cuda.amp import GradScaler, autocast
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
# from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import numpy as np

# # ==========================
# #        PARAMETERS
# # ==========================
# data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
# input_dim = 30  # Number of features
# latent_dim = 100
# batch_size = 256  # Increased batch size for GPU utilization
# epochs = 3000
# learning_rate = 0.0003
# betas = (0.5, 0.999)

# # ==========================
# #    DEVICE CONFIGURATION
# # ==========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # ==========================
# #      DATA PREPARATION
# # ==========================
# def load_data(path):
#     data = pd.read_csv(path)
#     return data

# def preprocess_data(data):
#     # Handle missing values
#     data = data.dropna()
    
#     # Features and target
#     X = data.drop('Class', axis=1)
#     y = data['Class']
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     return X_scaled, y

# def split_data(X, y, test_size=0.2, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y
#     )
#     return X_train, X_test, y_train, y_test

# # Execute preprocessing
# data = load_data(data_path)
# X, y = preprocess_data(data)
# X_train, X_test, y_train, y_test = split_data(X, y)

# # ==========================
# #      MODEL DEFINITIONS
# # ==========================
# # Improved Encoder
# class ImprovedEncoder(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedEncoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, latent_dim * 2)  # Mean and log-variance
#         )
    
#     def forward(self, x):
#         h = self.fc(x)
#         mu, logvar = torch.chunk(h, 2, dim=1)
#         return mu, logvar

# # Improved Decoder
# class ImprovedDecoder(nn.Module):
#     def __init__(self, latent_dim, output_dim):
#         super(ImprovedDecoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(latent_dim, 1024),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 2048),
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, output_dim),
#             nn.Sigmoid()
#         )
    
#     def forward(self, z):
#         return self.fc(z)

# # Improved Discriminator
# class ImprovedDiscriminator(nn.Module):
#     def __init__(self, input_dim):
#         super(ImprovedDiscriminator, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 1),
#             nn.Sigmoid()
#         )
    
#     def forward(self, x):
#         return self.fc(x)

# # Improved VAEGAN Model
# class ImprovedVAEGAN(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedVAEGAN, self).__init__()
#         self.encoder = ImprovedEncoder(input_dim, latent_dim)
#         self.decoder = ImprovedDecoder(latent_dim, input_dim)
#         self.discriminator = ImprovedDiscriminator(input_dim)
    
#     def reparameterize(self, mu, logvar):
#         std = torch.exp(0.5 * logvar)
#         eps = torch.randn_like(std)
#         return mu + eps * std
    
#     def forward(self, x):
#         mu, logvar = self.encoder(x)
#         z = self.reparameterize(mu, logvar)
#         reconstructed = self.decoder(z)
#         validity = self.discriminator(reconstructed)
#         return reconstructed, validity, mu, logvar

# # ==========================
# #      HELPER FUNCTIONS
# # ==========================
# def get_submodule(model, submodule_name):
#     """
#     Retrieve the submodule from the model, handling DataParallel if necessary.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodule_name (str): Name of the submodule to retrieve.
        
#     Returns:
#         torch.nn.Module: The requested submodule.
#     """
#     if isinstance(model, nn.DataParallel):
#         return getattr(model.module, submodule_name)
#     else:
#         return getattr(model, submodule_name)

# def get_parameters(model, submodules):
#     """
#     Retrieve the parameters from specified submodules.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodules (list of str): List of submodule names whose parameters are to be retrieved.
        
#     Returns:
#         list: List of parameters from the specified submodules.
#     """
#     params = []
#     for submodule in submodules:
#         sub = get_submodule(model, submodule)
#         params += list(sub.parameters())
#     return params

# # ==========================
# #      MODEL INITIALIZATION
# # ==========================
# # Initialize the model
# model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

# # Optionally use DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

# print("Model is on GPU:", next(model.parameters()).is_cuda)  # Debug statement
# print(model)  # Print model architecture

# # Function to count total parameters
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# total_params = count_parameters(model)
# print(f"Total trainable parameters: {total_params}")

# # ==========================
# #        DATA LOADING
# # ==========================
# # Convert training data to tensors
# train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=4  # Adjust based on your CPU cores
# )

# # ==========================
# #       OPTIMIZERS
# # ==========================
# # Initialize optimizers using helper functions
# optimizer_G = optim.Adam(
#     get_parameters(model, ['encoder', 'decoder']),
#     lr=learning_rate, betas=betas
# )
# optimizer_D = optim.Adam(
#     get_parameters(model, ['discriminator']),
#     lr=learning_rate, betas=betas
# )

# # ==========================
# #       LOSS FUNCTIONS
# # ==========================
# adversarial_loss = nn.MSELoss().to(device)
# reconstruction_loss = nn.MSELoss().to(device)
# kld_loss = lambda mu, logvar: -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# # ==========================
# #      MIXED PRECISION
# # ==========================
# scaler = GradScaler()

# # ==========================
# #      TRAINING LOOP
# # ==========================
# d_losses = []
# g_losses = []

# for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
#     epoch_d_loss = 0
#     epoch_g_loss = 0
#     model.train()
    
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
#         real_data = real_data.to(device)
#         batch_size_current = real_data.size(0)
        
#         # ===================
#         #   Train Discriminator
#         # ===================
#         optimizer_D.zero_grad()
#         with autocast():  # Enable autocasting for mixed precision
#             reconstructed, _, _, _ = model(real_data)
#             valid = torch.ones(batch_size_current, 1, device=device)
#             fake = torch.zeros(batch_size_current, 1, device=device)
            
#             # Use helper function to access discriminator
#             discriminator = get_submodule(model, 'discriminator')
#             real_discrim = discriminator(real_data)
#             fake_discrim = discriminator(reconstructed.detach())
            
#             real_loss = adversarial_loss(real_discrim, valid)
#             fake_loss = adversarial_loss(fake_discrim, fake)
#             d_loss = (real_loss + fake_loss) / 2
#         scaler.scale(d_loss).backward()
#         scaler.step(optimizer_D)
#         scaler.update()
        
#         # ===================
#         #    Train Generator
#         # ===================
#         optimizer_G.zero_grad()
#         with autocast():
#             reconstructed, validity, mu, logvar = model(real_data)
#             g_adv = adversarial_loss(validity, valid)
#             g_recon = reconstruction_loss(reconstructed, real_data)
#             g_kld = kld_loss(mu, logvar)
#             g_loss = g_adv + g_recon + g_kld
#         scaler.scale(g_loss).backward()
#         scaler.step(optimizer_G)
#         scaler.update()
        
#         # Accumulate losses
#         epoch_d_loss += d_loss.item()
#         epoch_g_loss += g_loss.item()
    
#     # Average losses for the epoch
#     avg_d_loss = epoch_d_loss / len(train_loader)
#     avg_g_loss = epoch_g_loss / len(train_loader)
    
#     d_losses.append(avg_d_loss)
#     g_losses.append(avg_g_loss)
    
#     # Logging every 100 epochs
#     if epoch % 100 == 0:
#         print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# # ==========================
# #        SAVE MODEL
# # ==========================
# # Save the model
# if isinstance(model, nn.DataParallel):
#     torch.save(model.module.state_dict(), 'improved_vae_gan.pth')
# else:
#     torch.save(model.state_dict(), 'improved_vae_gan.pth')

# # ==========================
# #        EVALUATION
# # ==========================
# # Parameters for Evaluation
# batch_size_eval = 256  # Increased batch size

# # Convert training data to tensors
# train_dataset_eval = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader_eval = DataLoader(
#     train_dataset_eval,
#     batch_size=batch_size_eval,
#     shuffle=False,  # Typically, shuffle=False for evaluation
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=4  # Adjust based on your CPU cores
# )

# # Load trained VAEGAN model and move to device
# model_eval = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
# if isinstance(model, nn.DataParallel):
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth'))
# else:
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth', map_location=device))
# model_eval.eval()
# print("Model loaded on GPU:", next(model_eval.parameters()).is_cuda)  # Debug statement

# # Generate synthetic data with tqdm
# synthetic_data = []
# with torch.no_grad():
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader_eval, desc="Generating Synthetic Data")):
#         real_data = real_data.to(device)
#         mu, logvar = get_submodule(model_eval, 'encoder')(real_data)
#         z = model_eval.reparameterize(mu, logvar)
#         gen_data = get_submodule(model_eval, 'decoder')(z)
#         synthetic_data.append(gen_data.cpu().numpy())
# synthetic_data = np.vstack(synthetic_data)

# # Combine real and synthetic data
# X_augmented = np.vstack((X_train, synthetic_data))
# y_augmented = np.hstack((y_train, np.ones(synthetic_data.shape[0])))

# # Split augmented data
# X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# # ==========================
# #    CLASSIFICATION MODELS
# # ==========================
# # Train DNN Classifier
# dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)  # Increased hidden layers
# dnn.fit(X_aug_train, y_aug_train)
# y_pred_dnn = dnn.predict(X_test)

# # Train XGBoost Classifier
# xgb_model = xgb.XGBClassifier(
#     use_label_encoder=False, 
#     eval_metric='logloss', 
#     n_estimators=100, 
#     max_depth=6, 
#     random_state=42
# )
# xgb_model.fit(X_train, y_train)
# y_pred_xgb = xgb_model.predict(X_test)

# # ==========================
# #          METRICS
# # ==========================
# # Metrics for DNN
# precision_dnn = precision_score(y_test, y_pred_dnn)
# recall_dnn = recall_score(y_test, y_pred_dnn)
# f1_dnn = f1_score(y_test, y_pred_dnn)

# # Metrics for XGBoost
# precision_xgb = precision_score(y_test, y_pred_xgb)
# recall_xgb = recall_score(y_test, y_pred_xgb)
# f1_xgb = f1_score(y_test, y_pred_xgb)

# # Print Classification Reports
# print("DNN Classification Report:")
# print(classification_report(y_test, y_pred_dnn))

# print("XGBoost Classification Report:")
# print(classification_report(y_test, y_pred_xgb))

# # Summary of Metrics
# summary_metrics = {
#     'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn},
#     'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb}
# }

# print("Summary of Classification Metrics:")
# for model_name, metrics in summary_metrics.items():
#     print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}")

In [ ]:
# # Improved version of above 
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch.cuda.amp import GradScaler, autocast
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
# from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import numpy as np

# # ==========================
# #        PARAMETERS
# # ==========================
# data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
# input_dim = 30  # Number of features
# latent_dim = 100
# batch_size = 1024  # Increased batch size for GPU utilization
# epochs = 3000
# learning_rate = 0.0003
# betas = (0.5, 0.999)

# # ==========================
# #    DEVICE CONFIGURATION
# # ==========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.backends.cudnn.benchmark = True  # Enable CuDNN benchmark for performance
# print(f"Using device: {device}")

# # ==========================
# #      DATA PREPARATION
# # ==========================
# def load_data(path):
#     data = pd.read_csv(path)
#     return data

# def preprocess_data(data):
#     # Handle missing values
#     data = data.dropna()
    
#     # Features and target
#     X = data.drop('Class', axis=1)
#     y = data['Class']
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     return X_scaled, y

# def split_data(X, y, test_size=0.2, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y
#     )
#     return X_train, X_test, y_train, y_test

# # Execute preprocessing
# data = load_data(data_path)
# X, y = preprocess_data(data)
# X_train, X_test, y_train, y_test = split_data(X, y)

# # ==========================
# #      MODEL DEFINITIONS
# # ==========================
# # Improved Encoder
# class ImprovedEncoder(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedEncoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, latent_dim * 2)  # Mean and log-variance
#         )
    
#     def forward(self, x):
#         h = self.fc(x)
#         mu, logvar = torch.chunk(h, 2, dim=1)
#         return mu, logvar

# # Improved Decoder
# class ImprovedDecoder(nn.Module):
#     def __init__(self, latent_dim, output_dim):
#         super(ImprovedDecoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(latent_dim, 1024),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 2048),
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, output_dim),
#             nn.Sigmoid()
#         )
    
#     def forward(self, z):
#         return self.fc(z)

# # Improved Discriminator
# class ImprovedDiscriminator(nn.Module):
#     def __init__(self, input_dim):
#         super(ImprovedDiscriminator, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 1),
#             nn.Sigmoid()
#         )
    
#     def forward(self, x):
#         return self.fc(x)

# # Improved VAEGAN Model
# class ImprovedVAEGAN(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedVAEGAN, self).__init__()
#         self.encoder = ImprovedEncoder(input_dim, latent_dim)
#         self.decoder = ImprovedDecoder(latent_dim, input_dim)
#         self.discriminator = ImprovedDiscriminator(input_dim)
    
#     def reparameterize(self, mu, logvar):
#         std = torch.exp(0.5 * logvar)
#         eps = torch.randn_like(std)
#         return mu + eps * std
    
#     def forward(self, x):
#         mu, logvar = self.encoder(x)
#         z = self.reparameterize(mu, logvar)
#         reconstructed = self.decoder(z)
#         validity = self.discriminator(reconstructed)
#         return reconstructed, validity, mu, logvar

# # ==========================
# #      HELPER FUNCTIONS
# # ==========================
# def get_submodule(model, submodule_name):
#     """
#     Retrieve the submodule from the model, handling DataParallel if necessary.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodule_name (str): Name of the submodule to retrieve.
        
#     Returns:
#         torch.nn.Module: The requested submodule.
#     """
#     if isinstance(model, nn.DataParallel):
#         return getattr(model.module, submodule_name)
#     else:
#         return getattr(model, submodule_name)

# def get_parameters(model, submodules):
#     """
#     Retrieve the parameters from specified submodules.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodules (list of str): List of submodule names whose parameters are to be retrieved.
        
#     Returns:
#         list: List of parameters from the specified submodules.
#     """
#     params = []
#     for submodule in submodules:
#         sub = get_submodule(model, submodule)
#         params += list(sub.parameters())
#     return params

# # ==========================
# #      MODEL INITIALIZATION
# # ==========================
# # Initialize the model
# model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

# # Optionally use DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

# print("Model is on GPU:", next(model.parameters()).is_cuda)  # Debug statement
# print(model)  # Print model architecture

# # Function to count total parameters
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# total_params = count_parameters(model)
# print(f"Total trainable parameters: {total_params}")

# # ==========================
# #        DATA LOADING
# # ==========================
# # Convert training data to tensors
# train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=12  # Increased number of workers for faster data loading
# )

# # ==========================
# #       OPTIMIZERS
# # ==========================
# # Initialize optimizers using helper functions
# optimizer_G = optim.Adam(
#     get_parameters(model, ['encoder', 'decoder']),
#     lr=learning_rate, betas=betas
# )
# optimizer_D = optim.Adam(
#     get_parameters(model, ['discriminator']),
#     lr=learning_rate, betas=betas
# )

# # ==========================
# #       LOSS FUNCTIONS
# # ==========================
# adversarial_loss = nn.MSELoss().to(device)
# reconstruction_loss = nn.MSELoss().to(device)
# kld_loss = lambda mu, logvar: -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# # ==========================
# #      MIXED PRECISION
# # ==========================
# scaler = GradScaler()

# # ==========================
# #      TRAINING LOOP
# # ==========================
# d_losses = []
# g_losses = []

# for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
#     epoch_d_loss = 0
#     epoch_g_loss = 0
#     model.train()
    
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
#         real_data = real_data.to(device)
#         batch_size_current = real_data.size(0)
        
#         # ===================
#         #   Train Discriminator
#         # ===================
#         optimizer_D.zero_grad()
#         with autocast():  # Enable autocasting for mixed precision
#             reconstructed, _, _, _ = model(real_data)
#             valid = torch.ones(batch_size_current, 1, device=device)
#             fake = torch.zeros(batch_size_current, 1, device=device)
            
#             # Use helper function to access discriminator
#             discriminator = get_submodule(model, 'discriminator')
#             real_discrim = discriminator(real_data)
#             fake_discrim = discriminator(reconstructed.detach())
            
#             real_loss = adversarial_loss(real_discrim, valid)
#             fake_loss = adversarial_loss(fake_discrim, fake)
#             d_loss = (real_loss + fake_loss) / 2
#         scaler.scale(d_loss).backward()
#         scaler.step(optimizer_D)
#         scaler.update()
        
#         # ===================
#         #    Train Generator
#         # ===================
#         optimizer_G.zero_grad()
#         with autocast():
#             reconstructed, validity, mu, logvar = model(real_data)
#             g_adv = adversarial_loss(validity, valid)
#             g_recon = reconstruction_loss(reconstructed, real_data)
#             g_kld = kld_loss(mu, logvar)
#             g_loss = g_adv + g_recon + g_kld
#         scaler.scale(g_loss).backward()
#         scaler.step(optimizer_G)
#         scaler.update()
        
#         # Accumulate losses
#         epoch_d_loss += d_loss.item()
#         epoch_g_loss += g_loss.item()
    
#     # Average losses for the epoch
#     avg_d_loss = epoch_d_loss / len(train_loader)
#     avg_g_loss = epoch_g_loss / len(train_loader)
    
#     d_losses.append(avg_d_loss)
#     g_losses.append(avg_g_loss)
    
#     # Logging every 100 epochs
#     if epoch % 100 == 0:
#         print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# # ==========================
# #        SAVE MODEL
# # ==========================
# # Save the model
# if isinstance(model, nn.DataParallel):
#     torch.save(model.module.state_dict(), 'improved_vae_gan.pth')
# else:
#     torch.save(model.state_dict(), 'improved_vae_gan.pth')

# # ==========================
# #        EVALUATION
# # ==========================
# # Parameters for Evaluation
# batch_size_eval = 1024  # Increased batch size

# # Convert training data to tensors
# train_dataset_eval = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader_eval = DataLoader(
#     train_dataset_eval,
#     batch_size=batch_size_eval,
#     shuffle=False,  # Typically, shuffle=False for evaluation
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=12  # Increased number of workers for faster data loading
# )

# # Load trained VAEGAN model and move to device
# model_eval = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
# if isinstance(model, nn.DataParallel):
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth'))
# else:
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth', map_location=device))
# model_eval.eval()
# print("Model loaded on GPU:", next(model_eval.parameters()).is_cuda)  # Debug statement

# # Generate synthetic data with tqdm
# synthetic_data = []
# with torch.no_grad():
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader_eval, desc="Generating Synthetic Data")):
#         real_data = real_data.to(device)
#         mu, logvar = get_submodule(model_eval, 'encoder')(real_data)
#         z = model_eval.reparameterize(mu, logvar)
#         gen_data = get_submodule(model_eval, 'decoder')(z)
#         synthetic_data.append(gen_data.cpu().numpy())
# synthetic_data = np.vstack(synthetic_data)

# # Combine real and synthetic data
# X_augmented = np.vstack((X_train, synthetic_data))
# y_augmented = np.hstack((y_train, np.ones(synthetic_data.shape[0])))

# # Split augmented data
# X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# # ==========================
# #    CLASSIFICATION MODELS
# # ==========================
# # Train DNN Classifier
# dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)  # Increased hidden layers
# dnn.fit(X_aug_train, y_aug_train)
# y_pred_dnn = dnn.predict(X_test)

# # Train XGBoost Classifier
# xgb_model = xgb.XGBClassifier(
#     use_label_encoder=False, 
#     eval_metric='logloss', 
#     n_estimators=100, 
#     max_depth=6, 
#     random_state=42
# )
# xgb_model.fit(X_train, y_train)
# y_pred_xgb = xgb_model.predict(X_test)

# # ==========================
# #          METRICS
# # ==========================
# # Metrics for DNN
# precision_dnn = precision_score(y_test, y_pred_dnn)
# recall_dnn = recall_score(y_test, y_pred_dnn)
# f1_dnn = f1_score(y_test, y_pred_dnn)

# # Metrics for XGBoost
# precision_xgb = precision_score(y_test, y_pred_xgb)
# recall_xgb = recall_score(y_test, y_pred_xgb)
# f1_xgb = f1_score(y_test, y_pred_xgb)

# # Print Classification Reports
# print("DNN Classification Report:")
# print(classification_report(y_test, y_pred_dnn))

# print("XGBoost Classification Report:")
# print(classification_report(y_test, y_pred_xgb))

# # Summary of Metrics
# summary_metrics = {
#     'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn},
#     'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb}
# }

# print("Summary of Classification Metrics:")
# for model_name, metrics in summary_metrics.items():
#     print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}")




# Even more improved version than above 

Improvements Implemented:
Further Increased Batch Size:

From: batch_size = 1024
To: batch_size = 4096
This allows processing more samples in parallel, maximizing GPU utilization across both T4 GPUs.

Further Increased Number of DataLoader Workers:

From: num_workers=12
To: num_workers=24
Increasing the number of workers enhances data loading speed, ensuring that the GPUs remain active without waiting for data.

Adjusted Evaluation Parameters:

Increased batch_size_eval to 4096
Increased num_workers to 24
Aligning evaluation parameters with training settings ensures consistency and maximizes the efficiency during the synthetic data generation phase.

Enhanced Discriminator Architecture:

Further Increased Layer Sizes in Discriminato

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch.cuda.amp import GradScaler, autocast
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
# from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import numpy as np

# # ==========================
# #        PARAMETERS
# # ==========================
# data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
# input_dim = 30  # Number of features
# latent_dim = 100
# batch_size = 4096  # Increased batch size for GPU utilization
# epochs = 3000
# learning_rate = 0.0003
# betas = (0.5, 0.999)

# # ==========================
# #    DEVICE CONFIGURATION
# # ==========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.backends.cudnn.benchmark = True  # Enable CuDNN benchmark for performance
# print(f"Using device: {device}")

# # ==========================
# #      DATA PREPARATION
# # ==========================
# def load_data(path):
#     data = pd.read_csv(path)
#     return data

# def preprocess_data(data):
#     # Handle missing values
#     data = data.dropna()
    
#     # Features and target
#     X = data.drop('Class', axis=1)
#     y = data['Class']
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     return X_scaled, y

# def split_data(X, y, test_size=0.2, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y
#     )
#     return X_train, X_test, y_train, y_test

# # Execute preprocessing
# data = load_data(data_path)
# X, y = preprocess_data(data)
# X_train, X_test, y_train, y_test = split_data(X, y)

# # ==========================
# #      MODEL DEFINITIONS
# # ==========================
# # Improved Encoder
# class ImprovedEncoder(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedEncoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, latent_dim * 2)  # Mean and log-variance
#         )
    
#     def forward(self, x):
#         h = self.fc(x)
#         mu, logvar = torch.chunk(h, 2, dim=1)
#         return mu, logvar

# # Improved Decoder
# class ImprovedDecoder(nn.Module):
#     def __init__(self, latent_dim, output_dim):
#         super(ImprovedDecoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(latent_dim, 1024),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 2048),
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, output_dim),
#             nn.Sigmoid()
#         )
    
#     def forward(self, z):
#         return self.fc(z)

# # Improved Discriminator
# class ImprovedDiscriminator(nn.Module):
#     def __init__(self, input_dim):
#         super(ImprovedDiscriminator, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 4096),  # Further increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(4096, 2048),
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1),
#             nn.Sigmoid()
#         )
    
#     def forward(self, x):
#         return self.fc(x)

# # Improved VAEGAN Model
# class ImprovedVAEGAN(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedVAEGAN, self).__init__()
#         self.encoder = ImprovedEncoder(input_dim, latent_dim)
#         self.decoder = ImprovedDecoder(latent_dim, input_dim)
#         self.discriminator = ImprovedDiscriminator(input_dim)
    
#     def reparameterize(self, mu, logvar):
#         std = torch.exp(0.5 * logvar)
#         eps = torch.randn_like(std)
#         return mu + eps * std
    
#     def forward(self, x):
#         mu, logvar = self.encoder(x)
#         z = self.reparameterize(mu, logvar)
#         reconstructed = self.decoder(z)
#         validity = self.discriminator(reconstructed)
#         return reconstructed, validity, mu, logvar

# # ==========================
# #      HELPER FUNCTIONS
# # ==========================
# def get_submodule(model, submodule_name):
#     """
#     Retrieve the submodule from the model, handling DataParallel if necessary.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodule_name (str): Name of the submodule to retrieve.
        
#     Returns:
#         torch.nn.Module: The requested submodule.
#     """
#     if isinstance(model, nn.DataParallel):
#         return getattr(model.module, submodule_name)
#     else:
#         return getattr(model, submodule_name)

# def get_parameters(model, submodules):
#     """
#     Retrieve the parameters from specified submodules.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodules (list of str): List of submodule names whose parameters are to be retrieved.
        
#     Returns:
#         list: List of parameters from the specified submodules.
#     """
#     params = []
#     for submodule in submodules:
#         sub = get_submodule(model, submodule)
#         params += list(sub.parameters())
#     return params

# # ==========================
# #      MODEL INITIALIZATION
# # ==========================
# # Initialize the model
# model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

# # Optionally use DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

# print("Model is on GPU:", next(model.parameters()).is_cuda)  # Debug statement
# print(model)  # Print model architecture

# # Function to count total parameters
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# total_params = count_parameters(model)
# print(f"Total trainable parameters: {total_params}")

# # ==========================
# #        DATA LOADING
# # ==========================
# # Convert training data to tensors
# train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=24  # Increased number of workers for faster data loading
# )

# # ==========================
# #       OPTIMIZERS
# # ==========================
# # Initialize optimizers using helper functions
# optimizer_G = optim.Adam(
#     get_parameters(model, ['encoder', 'decoder']),
#     lr=learning_rate, betas=betas
# )
# optimizer_D = optim.Adam(
#     get_parameters(model, ['discriminator']),
#     lr=learning_rate, betas=betas
# )

# # ==========================
# #       LOSS FUNCTIONS
# # ==========================
# adversarial_loss = nn.MSELoss().to(device)
# reconstruction_loss = nn.MSELoss().to(device)
# kld_loss = lambda mu, logvar: -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# # ==========================
# #      MIXED PRECISION
# # ==========================
# scaler = GradScaler()

# # ==========================
# #      TRAINING LOOP
# # ==========================
# d_losses = []
# g_losses = []

# for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
#     epoch_d_loss = 0
#     epoch_g_loss = 0
#     model.train()
    
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
#         real_data = real_data.to(device)
#         batch_size_current = real_data.size(0)
        
#         # ===================
#         #   Train Discriminator
#         # ===================
#         optimizer_D.zero_grad()
#         with autocast():  # Enable autocasting for mixed precision
#             reconstructed, _, _, _ = model(real_data)
#             valid = torch.ones(batch_size_current, 1, device=device)
#             fake = torch.zeros(batch_size_current, 1, device=device)
            
#             # Use helper function to access discriminator
#             discriminator = get_submodule(model, 'discriminator')
#             real_discrim = discriminator(real_data)
#             fake_discrim = discriminator(reconstructed.detach())
            
#             real_loss = adversarial_loss(real_discrim, valid)
#             fake_loss = adversarial_loss(fake_discrim, fake)
#             d_loss = (real_loss + fake_loss) / 2
#         scaler.scale(d_loss).backward()
#         scaler.step(optimizer_D)
#         scaler.update()
        
#         # ===================
#         #    Train Generator
#         # ===================
#         optimizer_G.zero_grad()
#         with autocast():
#             reconstructed, validity, mu, logvar = model(real_data)
#             g_adv = adversarial_loss(validity, valid)
#             g_recon = reconstruction_loss(reconstructed, real_data)
#             g_kld = kld_loss(mu, logvar)
#             g_loss = g_adv + g_recon + g_kld
#         scaler.scale(g_loss).backward()
#         scaler.step(optimizer_G)
#         scaler.update()
        
#         # Accumulate losses
#         epoch_d_loss += d_loss.item()
#         epoch_g_loss += g_loss.item()
    
#     # Average losses for the epoch
#     avg_d_loss = epoch_d_loss / len(train_loader)
#     avg_g_loss = epoch_g_loss / len(train_loader)
    
#     d_losses.append(avg_d_loss)
#     g_losses.append(avg_g_loss)
    
#     # Logging every 100 epochs
#     if epoch % 100 == 0:
#         print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# # ==========================
# #        SAVE MODEL
# # ==========================
# # Save the model
# if isinstance(model, nn.DataParallel):
#     torch.save(model.module.state_dict(), 'improved_vae_gan.pth')
# else:
#     torch.save(model.state_dict(), 'improved_vae_gan.pth')

# # ==========================
# #        EVALUATION
# # ==========================
# # Parameters for Evaluation
# batch_size_eval = 4096  # Further increased batch size

# # Convert training data to tensors
# train_dataset_eval = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader_eval = DataLoader(
#     train_dataset_eval,
#     batch_size=batch_size_eval,
#     shuffle=False,  # Typically, shuffle=False for evaluation
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=24  # Further increased number of workers for faster data loading
# )

# # Load trained VAEGAN model and move to device
# model_eval = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
# if isinstance(model, nn.DataParallel):
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth'))
# else:
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth', map_location=device))
# model_eval.eval()
# print("Model loaded on GPU:", next(model_eval.parameters()).is_cuda)  # Debug statement

# # Generate synthetic data with tqdm
# synthetic_data = []
# with torch.no_grad():
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader_eval, desc="Generating Synthetic Data")):
#         real_data = real_data.to(device)
#         mu, logvar = get_submodule(model_eval, 'encoder')(real_data)
#         z = model_eval.reparameterize(mu, logvar)
#         gen_data = get_submodule(model_eval, 'decoder')(z)
#         synthetic_data.append(gen_data.cpu().numpy())
# synthetic_data = np.vstack(synthetic_data)

# # Combine real and synthetic data
# X_augmented = np.vstack((X_train, synthetic_data))
# y_augmented = np.hstack((y_train, np.ones(synthetic_data.shape[0])))

# # Split augmented data
# X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# # ==========================
# #    CLASSIFICATION MODELS
# # ==========================
# # Train DNN Classifier
# dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)  # Increased hidden layers
# dnn.fit(X_aug_train, y_aug_train)
# y_pred_dnn = dnn.predict(X_test)

# # Train XGBoost Classifier
# xgb_model = xgb.XGBClassifier(
#     use_label_encoder=False, 
#     eval_metric='logloss', 
#     n_estimators=100, 
#     max_depth=6, 
#     random_state=42
# )
# xgb_model.fit(X_train, y_train)
# y_pred_xgb = xgb_model.predict(X_test)

# # ==========================
# #          METRICS
# # ==========================
# # Metrics for DNN
# precision_dnn = precision_score(y_test, y_pred_dnn)
# recall_dnn = recall_score(y_test, y_pred_dnn)
# f1_dnn = f1_score(y_test, y_pred_dnn)

# # Metrics for XGBoost
# precision_xgb = precision_score(y_test, y_pred_xgb)
# recall_xgb = recall_score(y_test, y_pred_xgb)
# f1_xgb = f1_score(y_test, y_pred_xgb)

# # Print Classification Reports
# print("DNN Classification Report:")
# print(classification_report(y_test, y_pred_dnn))

# print("XGBoost Classification Report:")
# print(classification_report(y_test, y_pred_xgb))

# # Summary of Metrics
# summary_metrics = {
#     'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn},
#     'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb}
# }

# print("Summary of Classification Metrics:")
# for model_name, metrics in summary_metrics.items():
#     print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}")




In [ ]:
# # Even more Advanced code 

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch.cuda.amp import GradScaler, autocast
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
# from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# import numpy as np

# # ==========================
# #        PARAMETERS
# # ==========================
# data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
# input_dim = 30  # Number of features
# latent_dim = 100
# batch_size = 8192  # Further increased batch size for GPU utilization
# epochs = 3000
# learning_rate = 0.0003
# betas = (0.5, 0.999)

# # ==========================
# #    DEVICE CONFIGURATION
# # ==========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.backends.cudnn.benchmark = True  # Enable CuDNN benchmark for performance
# print(f"Using device: {device}")

# # ==========================
# #      DATA PREPARATION
# # ==========================
# def load_data(path):
#     data = pd.read_csv(path)
#     return data

# def preprocess_data(data):
#     # Handle missing values
#     data = data.dropna()
    
#     # Features and target
#     X = data.drop('Class', axis=1)
#     y = data['Class']
    
#     # Normalize features
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)
    
#     return X_scaled, y

# def split_data(X, y, test_size=0.2, random_state=42):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=random_state, stratify=y
#     )
#     return X_train, X_test, y_train, y_test

# # Execute preprocessing
# data = load_data(data_path)
# X, y = preprocess_data(data)
# X_train, X_test, y_train, y_test = split_data(X, y)

# # ==========================
# #      MODEL DEFINITIONS
# # ==========================
# # Improved Encoder
# class ImprovedEncoder(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedEncoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 2048),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1024),
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, latent_dim * 2)  # Mean and log-variance
#         )
    
#     def forward(self, x):
#         h = self.fc(x)
#         mu, logvar = torch.chunk(h, 2, dim=1)
#         return mu, logvar

# # Improved Decoder
# class ImprovedDecoder(nn.Module):
#     def __init__(self, latent_dim, output_dim):
#         super(ImprovedDecoder, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(latent_dim, 1024),  # Increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(1024, 2048),
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, output_dim),
#             nn.Sigmoid()
#         )
    
#     def forward(self, z):
#         return self.fc(z)

# # Improved Discriminator
# class ImprovedDiscriminator(nn.Module):
#     def __init__(self, input_dim):
#         super(ImprovedDiscriminator, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, 4096),  # Further increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(4096, 2048),       # Further increased layer size
#             nn.ReLU(inplace=True),
#             nn.Linear(2048, 1),
#             nn.Sigmoid()
#         )
    
#     def forward(self, x):
#         return self.fc(x)

# # Improved VAEGAN Model
# class ImprovedVAEGAN(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(ImprovedVAEGAN, self).__init__()
#         self.encoder = ImprovedEncoder(input_dim, latent_dim)
#         self.decoder = ImprovedDecoder(latent_dim, input_dim)
#         self.discriminator = ImprovedDiscriminator(input_dim)
    
#     def reparameterize(self, mu, logvar):
#         std = torch.exp(0.5 * logvar)
#         eps = torch.randn_like(std)
#         return mu + eps * std
    
#     def forward(self, x):
#         mu, logvar = self.encoder(x)
#         z = self.reparameterize(mu, logvar)
#         reconstructed = self.decoder(z)
#         validity = self.discriminator(reconstructed)
#         return reconstructed, validity, mu, logvar

# # ==========================
# #      HELPER FUNCTIONS
# # ==========================
# def get_submodule(model, submodule_name):
#     """
#     Retrieve the submodule from the model, handling DataParallel if necessary.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodule_name (str): Name of the submodule to retrieve.
        
#     Returns:
#         torch.nn.Module: The requested submodule.
#     """
#     if isinstance(model, nn.DataParallel):
#         return getattr(model.module, submodule_name)
#     else:
#         return getattr(model, submodule_name)

# def get_parameters(model, submodules):
#     """
#     Retrieve the parameters from specified submodules.
    
#     Args:
#         model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
#         submodules (list of str): List of submodule names whose parameters are to be retrieved.
        
#     Returns:
#         list: List of parameters from the specified submodules.
#     """
#     params = []
#     for submodule in submodules:
#         sub = get_submodule(model, submodule)
#         params += list(sub.parameters())
#     return params

# # ==========================
# #      MODEL INITIALIZATION
# # ==========================
# # Initialize the model
# model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

# # Optionally use DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

# print("Model is on GPU:", next(model.parameters()).is_cuda)  # Debug statement
# print(model)  # Print model architecture

# # Function to count total parameters
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# total_params = count_parameters(model)
# print(f"Total trainable parameters: {total_params}")

# # ==========================
# #        DATA LOADING
# # ==========================
# # Convert training data to tensors
# train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=48  # Further increased number of workers for faster data loading
# )

# # ==========================
# #       OPTIMIZERS
# # ==========================
# # Initialize optimizers using helper functions
# optimizer_G = optim.Adam(
#     get_parameters(model, ['encoder', 'decoder']),
#     lr=learning_rate, betas=betas
# )
# optimizer_D = optim.Adam(
#     get_parameters(model, ['discriminator']),
#     lr=learning_rate, betas=betas
# )

# # ==========================
# #       LOSS FUNCTIONS
# # ==========================
# adversarial_loss = nn.MSELoss().to(device)
# reconstruction_loss = nn.MSELoss().to(device)
# kld_loss = lambda mu, logvar: -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# # ==========================
# #      MIXED PRECISION
# # ==========================
# scaler = GradScaler()

# # ==========================
# #      TRAINING LOOP
# # ==========================
# d_losses = []
# g_losses = []

# for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
#     epoch_d_loss = 0
#     epoch_g_loss = 0
#     model.train()
    
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
#         real_data = real_data.to(device)
#         batch_size_current = real_data.size(0)
        
#         # ===================
#         #   Train Discriminator
#         # ===================
#         optimizer_D.zero_grad()
#         with autocast():  # Enable autocasting for mixed precision
#             reconstructed, _, _, _ = model(real_data)
#             valid = torch.ones(batch_size_current, 1, device=device)
#             fake = torch.zeros(batch_size_current, 1, device=device)
            
#             # Use helper function to access discriminator
#             discriminator = get_submodule(model, 'discriminator')
#             real_discrim = discriminator(real_data)
#             fake_discrim = discriminator(reconstructed.detach())
            
#             real_loss = adversarial_loss(real_discrim, valid)
#             fake_loss = adversarial_loss(fake_discrim, fake)
#             d_loss = (real_loss + fake_loss) / 2
#         scaler.scale(d_loss).backward()
#         scaler.step(optimizer_D)
#         scaler.update()
        
#         # ===================
#         #    Train Generator
#         # ===================
#         optimizer_G.zero_grad()
#         with autocast():
#             reconstructed, validity, mu, logvar = model(real_data)
#             g_adv = adversarial_loss(validity, valid)
#             g_recon = reconstruction_loss(reconstructed, real_data)
#             g_kld = kld_loss(mu, logvar)
#             g_loss = g_adv + g_recon + g_kld
#         scaler.scale(g_loss).backward()
#         scaler.step(optimizer_G)
#         scaler.update()
        
#         # Accumulate losses
#         epoch_d_loss += d_loss.item()
#         epoch_g_loss += g_loss.item()
    
#     # Average losses for the epoch
#     avg_d_loss = epoch_d_loss / len(train_loader)
#     avg_g_loss = epoch_g_loss / len(train_loader)
    
#     d_losses.append(avg_d_loss)
#     g_losses.append(avg_g_loss)
    
#     # Logging every 100 epochs
#     if epoch % 100 == 0:
#         print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# # ==========================
# #        SAVE MODEL
# # ==========================
# # Save the model
# if isinstance(model, nn.DataParallel):
#     torch.save(model.module.state_dict(), 'improved_vae_gan.pth')
# else:
#     torch.save(model.state_dict(), 'improved_vae_gan.pth')

# # ==========================
# #        EVALUATION
# # ==========================
# # Parameters for Evaluation
# batch_size_eval = 8192  # Further increased batch size

# # Convert training data to tensors
# train_dataset_eval = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
# train_loader_eval = DataLoader(
#     train_dataset_eval,
#     batch_size=batch_size_eval,
#     shuffle=False,  # Typically, shuffle=False for evaluation
#     pin_memory=True if device.type == 'cuda' else False,
#     num_workers=48  # Further increased number of workers for faster data loading
# )

# # Load trained VAEGAN model and move to device
# model_eval = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
# if isinstance(model, nn.DataParallel):
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth'))
# else:
#     model_eval.load_state_dict(torch.load('improved_vae_gan.pth', map_location=device))
# model_eval.eval()
# print("Model loaded on GPU:", next(model_eval.parameters()).is_cuda)  # Debug statement

# # Generate synthetic data with tqdm
# synthetic_data = []
# with torch.no_grad():
#     for batch_idx, (real_data, _) in enumerate(tqdm(train_loader_eval, desc="Generating Synthetic Data")):
#         real_data = real_data.to(device)
#         mu, logvar = get_submodule(model_eval, 'encoder')(real_data)
#         z = model_eval.reparameterize(mu, logvar)
#         gen_data = get_submodule(model_eval, 'decoder')(z)
#         synthetic_data.append(gen_data.cpu().numpy())
# synthetic_data = np.vstack(synthetic_data)

# # Combine real and synthetic data
# X_augmented = np.vstack((X_train, synthetic_data))
# y_augmented = np.hstack((y_train, np.ones(synthetic_data.shape[0])))

# # Split augmented data
# X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# # ==========================
# #    CLASSIFICATION MODELS
# # ==========================
# # Train DNN Classifier
# dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)  # Increased hidden layers
# dnn.fit(X_aug_train, y_aug_train)
# y_pred_dnn = dnn.predict(X_test)

# # Train XGBoost Classifier
# xgb_model = xgb.XGBClassifier(
#     use_label_encoder=False, 
#     eval_metric='logloss', 
#     n_estimators=100, 
#     max_depth=6, 
#     random_state=42
# )
# xgb_model.fit(X_train, y_train)
# y_pred_xgb = xgb_model.predict(X_test)

# # ==========================
# #          METRICS
# # ==========================
# # Metrics for DNN
# precision_dnn = precision_score(y_test, y_pred_dnn)
# recall_dnn = recall_score(y_test, y_pred_dnn)
# f1_dnn = f1_score(y_test, y_pred_dnn)

# # Metrics for XGBoost
# precision_xgb = precision_score(y_test, y_pred_xgb)
# recall_xgb = recall_score(y_test, y_pred_xgb)
# f1_xgb = f1_score(y_test, y_pred_xgb)

# # Print Classification Reports
# print("DNN Classification Report:")
# print(classification_report(y_test, y_pred_dnn))

# print("XGBoost Classification Report:")
# print(classification_report(y_test, y_pred_xgb))

# # Summary of Metrics
# summary_metrics = {
#     'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn},
#     'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb}
# }

# print("Summary of Classification Metrics:")
# for model_name, metrics in summary_metrics.items():
#     print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}")




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# ==========================
#        PARAMETERS
# ==========================
data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
input_dim = 30  # Number of features
latent_dim = 100
batch_size = 8192  # Further increased batch size for GPU utilization
epochs = 3000
learning_rate = 0.0003
betas = (0.5, 0.999)

# ==========================
#    DEVICE CONFIGURATION
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # Enable CuDNN benchmark for performance
print(f"Using device: {device}")

# ==========================
#      DATA PREPARATION
# ==========================
def load_data(path):
    data = pd.read_csv(path)
    return data

def preprocess_data(data):
    # Handle missing values
    data = data.dropna()
    
    # Features and target
    X = data.drop('Class', axis=1)
    y = data['Class']
    
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y

def split_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

# Execute preprocessing
data = load_data(data_path)
X, y = preprocess_data(data)
X_train, X_test, y_train, y_test = split_data(X, y)

# ==========================
#      MODEL DEFINITIONS
# ==========================
# Improved Encoder
class ImprovedEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(ImprovedEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 4096),  # Increased layer size
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, latent_dim * 2)  # Mean and log-variance
        )
    
    def forward(self, x):
        h = self.fc(x)
        mu, logvar = torch.chunk(h, 2, dim=1)
        return mu, logvar

# Improved Decoder
class ImprovedDecoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(ImprovedDecoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 2048),  # Increased layer size
            nn.ReLU(inplace=True),
            nn.Linear(2048, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
            nn.Sigmoid()
        )
    
    def forward(self, z):
        return self.fc(z)

# Improved Discriminator
class ImprovedDiscriminator(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedDiscriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 8192),  # Further increased layer size
            nn.ReLU(inplace=True),
            nn.Linear(8192, 4096),       # Further increased layer size
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.fc(x)

# Improved VAEGAN Model
class ImprovedVAEGAN(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(ImprovedVAEGAN, self).__init__()
        self.encoder = ImprovedEncoder(input_dim, latent_dim)
        self.decoder = ImprovedDecoder(latent_dim, input_dim)
        self.discriminator = ImprovedDiscriminator(input_dim)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        validity = self.discriminator(reconstructed)
        return reconstructed, validity, mu, logvar

# ==========================
#      HELPER FUNCTIONS
# ==========================
def get_submodule(model, submodule_name):
    """
    Retrieve the submodule from the model, handling DataParallel if necessary.
    
    Args:
        model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
        submodule_name (str): Name of the submodule to retrieve.
        
    Returns:
        torch.nn.Module: The requested submodule.
    """
    if isinstance(model, nn.DataParallel):
        return getattr(model.module, submodule_name)
    else:
        return getattr(model, submodule_name)

def get_parameters(model, submodules):
    """
    Retrieve the parameters from specified submodules.
    
    Args:
        model (torch.nn.Module): The main model, possibly wrapped with DataParallel.
        submodules (list of str): List of submodule names whose parameters are to be retrieved.
        
    Returns:
        list: List of parameters from the specified submodules.
    """
    params = []
    for submodule in submodules:
        sub = get_submodule(model, submodule)
        params += list(sub.parameters())
    return params

# ==========================
#      MODEL INITIALIZATION
# ==========================
# Initialize the model
model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

# Optionally use DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model, device_ids=[0,1])

print("Model is on GPU:", next(model.parameters()).is_cuda)  # Debug statement
print(model)  # Print model architecture

# Function to count total parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params}")

# ==========================
#        DATA LOADING
# ==========================
# Convert training data to tensors
train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True if device.type == 'cuda' else False,
    num_workers=48,  # Further increased number of workers for faster data loading
    prefetch_factor=4  # Added prefetch_factor for better data loading performance
)

# ==========================
#       OPTIMIZERS
# ==========================
# Initialize optimizers using helper functions
optimizer_G = optim.Adam(
    get_parameters(model, ['encoder', 'decoder']),
    lr=learning_rate, betas=betas
)
optimizer_D = optim.Adam(
    get_parameters(model, ['discriminator']),
    lr=learning_rate, betas=betas
)

# ==========================
#       LOSS FUNCTIONS
# ==========================
adversarial_loss = nn.MSELoss().to(device)
reconstruction_loss = nn.MSELoss().to(device)
kld_loss = lambda mu, logvar: -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# ==========================
#      MIXED PRECISION
# ==========================
scaler = GradScaler()

# ==========================
#      TRAINING LOOP
# ==========================
d_losses = []
g_losses = []

for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
    epoch_d_loss = 0
    epoch_g_loss = 0
    model.train()
    
    for batch_idx, (real_data, _) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
        real_data = real_data.to(device)
        batch_size_current = real_data.size(0)
        
        # ===================
        #   Train Discriminator
        # ===================
        optimizer_D.zero_grad()
        with autocast():  # Enable autocasting for mixed precision
            reconstructed, _, _, _ = model(real_data)
            valid = torch.ones(batch_size_current, 1, device=device)
            fake = torch.zeros(batch_size_current, 1, device=device)
            
            # Use helper function to access discriminator
            discriminator = get_submodule(model, 'discriminator')
            real_discrim = discriminator(real_data)
            fake_discrim = discriminator(reconstructed.detach())
            
            real_loss = adversarial_loss(real_discrim, valid)
            fake_loss = adversarial_loss(fake_discrim, fake)
            d_loss = (real_loss + fake_loss) / 2
        scaler.scale(d_loss).backward()
        scaler.step(optimizer_D)
        scaler.update()
        
        # ===================
        #    Train Generator
        # ===================
        optimizer_G.zero_grad()
        with autocast():
            reconstructed, validity, mu, logvar = model(real_data)
            g_adv = adversarial_loss(validity, valid)
            g_recon = reconstruction_loss(reconstructed, real_data)
            g_kld = kld_loss(mu, logvar)
            g_loss = g_adv + g_recon + g_kld
        scaler.scale(g_loss).backward()
        scaler.step(optimizer_G)
        scaler.update()
        
        # Accumulate losses
        epoch_d_loss += d_loss.item()
        epoch_g_loss += g_loss.item()
    
    # Average losses for the epoch
    avg_d_loss = epoch_d_loss / len(train_loader)
    avg_g_loss = epoch_g_loss / len(train_loader)
    
    d_losses.append(avg_d_loss)
    g_losses.append(avg_g_loss)
    
    # Logging every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")

# ==========================
#        SAVE MODEL
# ==========================
# Save the model
if isinstance(model, nn.DataParallel):
    torch.save(model.module.state_dict(), 'improved_vae_gan.pth')
else:
    torch.save(model.state_dict(), 'improved_vae_gan.pth')

# ==========================
#        EVALUATION
# ==========================
# Parameters for Evaluation
batch_size_eval = 8192  # Further increased batch size

# Convert training data to tensors
train_dataset_eval = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader_eval = DataLoader(
    train_dataset_eval,
    batch_size=batch_size_eval,
    shuffle=False,  # Typically, shuffle=False for evaluation
    pin_memory=True if device.type == 'cuda' else False,
    num_workers=48,  # Further increased number of workers for faster data loading
    prefetch_factor=4  # Added prefetch_factor for better data loading performance
)

# Load trained VAEGAN model and move to device
model_eval = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)
if isinstance(model, nn.DataParallel):
    model_eval.load_state_dict(torch.load('improved_vae_gan.pth'))
    model_eval = nn.DataParallel(model_eval, device_ids=[0,1])
else:
    model_eval.load_state_dict(torch.load('improved_vae_gan.pth', map_location=device))
print("Model loaded on GPU:", next(model_eval.parameters()).is_cuda)  # Debug statement

# Generate synthetic data with tqdm
synthetic_data = []
with torch.no_grad():
    for batch_idx, (real_data, _) in enumerate(tqdm(train_loader_eval, desc="Generating Synthetic Data")):
        real_data = real_data.to(device)
        mu, logvar = get_submodule(model_eval, 'encoder')(real_data)
        z = model_eval.module.reparameterize(mu, logvar) if isinstance(model_eval, nn.DataParallel) else model_eval.reparameterize(mu, logvar)
        gen_data = get_submodule(model_eval, 'decoder')(z)
        synthetic_data.append(gen_data.cpu().numpy())
synthetic_data = np.vstack(synthetic_data)

# Combine real and synthetic data
X_augmented = np.vstack((X_train, synthetic_data))
y_augmented = np.hstack((y_train, np.ones(synthetic_data.shape[0])))

# Split augmented data
X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# ==========================
#    CLASSIFICATION MODELS
# ==========================
# Train DNN Classifier
dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)  # Increased hidden layers
dnn.fit(X_aug_train, y_aug_train)
y_pred_dnn = dnn.predict(X_test)

# Train XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False, 
    eval_metric='logloss', 
    n_estimators=100, 
    max_depth=6, 
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# ==========================
#          METRICS
# ==========================
# Metrics for DNN
precision_dnn = precision_score(y_test, y_pred_dnn)
recall_dnn = recall_score(y_test, y_pred_dnn)
f1_dnn = f1_score(y_test, y_pred_dnn)

# Metrics for XGBoost
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

# Print Classification Reports
print("DNN Classification Report:")
print(classification_report(y_test, y_pred_dnn))

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

# Summary of Metrics
summary_metrics = {
    'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn},
    'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb}
}

print("Summary of Classification Metrics:")
for model_name, metrics in summary_metrics.items():
    print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}")




In [ ]:
# Improved code --> latest code with features like 
'''

1.Add Attention Mechanisms:
    Implement self-attention layers in the encoder and decoder to enhance feature representation.
2.Implement Residual Connections:
    Incorporate residual blocks within the encoder and decoder to facilitate deeper architectures.
3.Explore Different Fusion Strategies:
    Replace simple averaging with concatenation followed by a dense layer for merging encoder outputs.
4.Introduce Dropout Layers:
    Add dropout layers to prevent overfitting in the encoder, decoder, and discriminator.
5.Use Alternative Loss Functions:
    Integrate Wasserstein loss with gradient penalty for improved GAN training stability.
6.Integrate Batch Normalization:
    Apply batch normalization after linear layers to stabilize and accelerate training.
7.Expand Latent Space Dimensionality:
    Increase the latent dimension to capture more complex data distributions.
8.Incorporate Conditional GANs:
    Add label conditioning to the GAN to generate controlled synthetic samples.
9.Enhance Data Augmentation:
    Apply advanced data augmentation techniques like SMOTE or ADASYN.
10.Optimize Training Strategies:
    Use AdamW optimizer and implement learning rate schedulers for better convergence.
11.Implement Early Stopping and Checkpointing:
    Add early stopping to prevent overfitting and save the best model checkpoints.
12.Evaluate with Diverse Metrics:
    Include ROC-AUC and Precision-Recall AUC in the evaluation metrics.
13.Parallelize Data Processing:
    Optimize DataLoader settings to fully utilize CPU cores.
14.Leverage Transfer Learning:
    Utilize a pre-trained model for the discriminator to benefit from learned features.
15.Apply Gradient Penalty:
    Incorporate gradient penalty in the discriminator loss to improve GAN training stability.

'''
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, roc_auc_score, precision_recall_curve
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
from tqdm import tqdm
from models.encoder import ImprovedEncoder
from models.decoder import ImprovedDecoder
from models.discriminator import ImprovedDiscriminator
from utils.early_stopping import EarlyStopping

# ==========================
#        PARAMETERS
# ==========================
data_path = '/kaggle/input/creditcardfraud/creditcard.csv'  # Replace with your dataset path
input_dim = 30  # Number of features
latent_dim = 200  # Expanded latent dimension
num_classes = 2  # For conditional GAN
batch_size = 8192
epochs = 3000
learning_rate = 0.0003
betas = (0.5, 0.999)
patience = 50  # For early stopping

# ==========================
#    DEVICE CONFIGURATION
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print(f"Using device: {device}")

# ==========================
#      DATA PREPARATION
# ==========================
def load_data(path):
    data = pd.read_csv(path)
    return data

def preprocess_data(data):
    data = data.dropna()
    X = data.drop('Class', axis=1)
    y = data['Class']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y

def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

data = load_data(data_path)
X, y = preprocess_data(data)
X_train, X_test, y_train, y_test = split_data(X, y)

# ==========================
#      DATA AUGMENTATION
# ==========================
from imblearn.over_sampling import ADASYN

def augment_data(X, y):
    sampler = ADASYN(random_state=42)
    X_res, y_res = sampler.fit_resample(X, y)
    return X_res, y_res

X_train_aug, y_train_aug = augment_data(X_train, y_train)

# ==========================
#      MODEL DEFINITIONS
# ==========================
model = ImprovedVAEGAN(input_dim=input_dim, latent_dim=latent_dim).to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

print("Model is on GPU:", next(model.parameters()).is_cuda)
print(model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {count_parameters(model)}")

# ==========================
#        DATA LOADING
# ==========================
train_dataset = TensorDataset(torch.FloatTensor(X_train_aug), torch.LongTensor(y_train_aug))
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True if device.type == 'cuda' else False,
    num_workers=16,
    prefetch_factor=2
)

# ==========================
#       OPTIMIZERS & SCHEDULERS
# ==========================
optimizer_G = optim.AdamW(
    list(model.module.encoder.parameters()) + list(model.module.decoder.parameters()),
    lr=learning_rate, betas=betas
)
optimizer_D = optim.AdamW(
    model.module.discriminator.parameters(),
    lr=learning_rate, betas=betas
)

scheduler_G = optim.lr_scheduler.StepLR(optimizer_G, step_size=1000, gamma=0.5)
scheduler_D = optim.lr_scheduler.StepLR(optimizer_D, step_size=1000, gamma=0.5)

# ==========================
#       LOSS FUNCTIONS
# ==========================
adversarial_loss = nn.BCELoss().to(device)
reconstruction_loss = nn.MSELoss().to(device)

def gradient_penalty(discriminator, real_samples, fake_samples, labels):
    alpha = torch.randn(real_samples.size(0), 1).to(device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = discriminator(interpolates, labels)
    fake = torch.ones(real_samples.size(0), 1).to(device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return penalty

# ==========================
#      MIXED PRECISION
# ==========================
scaler = GradScaler()

# ==========================
#   EARLY STOPPING
# ==========================
early_stopping = EarlyStopping(patience=patience, verbose=True)

# ==========================
#      TRAINING LOOP
# ==========================
d_losses = []
g_losses = []

for epoch in tqdm(range(1, epochs + 1), desc="Training Epochs"):
    epoch_d_loss = 0
    epoch_g_loss = 0
    model.train()
    
    for batch_idx, (real_data, labels) in enumerate(tqdm(train_loader, desc="Batches", leave=False)):
        real_data = real_data.to(device)
        labels = labels.to(device)
        batch_size_current = real_data.size(0)
        
        # ===================
        #   Train Discriminator
        # ===================
        optimizer_D.zero_grad()
        with autocast():
            reconstructed, _, mu, logvar = model(real_data)
            valid = torch.ones(batch_size_current, 1, device=device)
            fake = torch.zeros(batch_size_current, 1, device=device)
            
            real_validity = model.module.discriminator(real_data, labels)
            fake_validity = model.module.discriminator(reconstructed.detach(), labels)
            
            d_real_loss = adversarial_loss(real_validity, valid)
            d_fake_loss = adversarial_loss(fake_validity, fake)
            gp = gradient_penalty(model.module.discriminator, real_data, reconstructed, labels)
            d_loss = d_real_loss + d_fake_loss + 10 * gp
        scaler.scale(d_loss).backward()
        scaler.step(optimizer_D)
        scaler.update()
        
        # ===================
        #    Train Generator
        # ===================
        optimizer_G.zero_grad()
        with autocast():
            reconstructed, validity, mu, logvar = model(real_data)
            g_adv = adversarial_loss(validity, valid)
            g_recon = reconstruction_loss(reconstructed, real_data)
            g_kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
            g_loss = g_adv + g_recon + g_kld
        scaler.scale(g_loss).backward()
        scaler.step(optimizer_G)
        scaler.update()
        
        epoch_d_loss += d_loss.item()
        epoch_g_loss += g_loss.item()
    
    scheduler_D.step()
    scheduler_G.step()
    
    avg_d_loss = epoch_d_loss / len(train_loader)
    avg_g_loss = epoch_g_loss / len(train_loader)
    
    d_losses.append(avg_d_loss)
    g_losses.append(avg_g_loss)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f}")
    
    # Early Stopping
    early_stopping(avg_g_loss, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# ==========================
#        SAVE MODEL
# ==========================
torch.save(model.module.state_dict(), 'improved_vae_gan.pth')

# ==========================
#        EVALUATION
# ==========================
model.eval()
synthetic_data = []
synthetic_labels = []

with torch.no_grad():
    for real_data, labels in tqdm(train_loader, desc="Generating Synthetic Data"):
        real_data = real_data.to(device)
        labels = labels.to(device)
        mu, logvar = model.module.encoder(real_data)
        z = model.module.reparameterize(mu, logvar)
        gen_data = model.module.decoder(z)
        synthetic_data.append(gen_data.cpu().numpy())
        synthetic_labels.append(labels.cpu().numpy())

synthetic_data = np.vstack(synthetic_data)
synthetic_labels = np.hstack(synthetic_labels)

X_augmented = np.vstack((X_train, synthetic_data))
y_augmented = np.hstack((y_train, synthetic_labels))

X_aug_train, X_aug_test, y_aug_train, y_aug_test = split_data(X_augmented, y_augmented)

# ==========================
#    CLASSIFICATION MODELS
# ==========================
# Train DNN Classifier
dnn = MLPClassifier(hidden_layer_sizes=(200,), max_iter=300, random_state=42)
dnn.fit(X_aug_train, y_aug_train)
y_pred_dnn = dnn.predict(X_test)
y_prob_dnn = dnn.predict_proba(X_test)[:,1]

# Train XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False, 
    eval_metric='logloss', 
    n_estimators=100, 
    max_depth=6, 
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:,1]

# ==========================
#          METRICS
# ==========================
# Metrics for DNN
precision_dnn = precision_score(y_test, y_pred_dnn)
recall_dnn = recall_score(y_test, y_pred_dnn)
f1_dnn = f1_score(y_test, y_pred_dnn)
roc_auc_dnn = roc_auc_score(y_test, y_prob_dnn)

# Metrics for XGBoost
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_prob_xgb)

# Print Classification Reports
print("DNN Classification Report:")
print(classification_report(y_test, y_pred_dnn))

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

# Summary of Metrics
summary_metrics = {
    'DNN': {'Precision': precision_dnn, 'Recall': recall_dnn, 'F1-Score': f1_dnn, 'ROC-AUC': roc_auc_dnn},
    'XGBoost': {'Precision': precision_xgb, 'Recall': recall_xgb, 'F1-Score': f1_xgb, 'ROC-AUC': roc_auc_xgb}
}

print("Summary of Classification Metrics:")
for model_name, metrics in summary_metrics.items():
    print(f"{model_name}: Precision={metrics['Precision']:.4f}, Recall={metrics['Recall']:.4f}, F1-Score={metrics['F1-Score']:.4f}, ROC-AUC={metrics['ROC-AUC']:.4f}")
    
# Plot ROC Curves
from sklearn.metrics import roc_curve

fpr_dnn, tpr_dnn, _ = roc_curve(y_test, y_prob_dnn)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_prob_xgb)

plt.figure(figsize=(10,7))
plt.plot(fpr_dnn, tpr_dnn, label=f'DNN ROC AUC = {roc_auc_dnn:.4f}')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost ROC AUC = {roc_auc_xgb:.4f}')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.show()

# Plot Precision-Recall Curves
precision_dnn_vals, recall_dnn_vals, _ = precision_recall_curve(y_test, y_prob_dnn)
precision_xgb_vals, recall_xgb_vals, _ = precision_recall_curve(y_test, y_prob_xgb)

plt.figure(figsize=(10,7))
plt.plot(recall_dnn_vals, precision_dnn_vals, label=f'DNN PR AUC = {roc_auc_dnn:.4f}')
plt.plot(recall_xgb_vals, precision_xgb_vals, label=f'XGBoost PR AUC = {roc_auc_xgb:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
plt.show()